In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.impute import KNNImputer
import numpy as np
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import roc_auc_score, confusion_matrix
import seaborn as sns
from random import sample, seed 
from scipy import stats

import matplotlib.pyplot as plt
from gseapy.plot import barplot, dotplot
from mycolorpy import colorlist as mcp
from matplotlib.lines import Line2D

from sklearn.preprocessing import MinMaxScaler



In [ ]:
#Read omics data
omics_prot_path = '/data/sharedData/UK_BIOBANK_DATA/Download_Data/ProteomicsData/SecondPhase/Olink_proteomics_data_2ndPhase_transposed_decoded2UNIportID.txt'
omics_met_path = '/data/sharedData/UK_BIOBANK_DATA/Download_Data/Metabolomics_Data/SecondPhaseData/nmr_biomarker_data_RemovedTechVariation_secondPHASE_FilteredDuplicateV2.csv'

omics_met = pd.read_csv(omics_met_path, sep = ',', index_col = 0)
omics_prot = pd.read_csv(omics_prot_path, sep = '\t', index_col = 0)



In [ ]:
data_statistics = ({'Disease':[],
                    'Data':[],
                    'Count':[],
                    'Age_median':[],
                    'Age_IQR_low': [],
                    'Age_IQR_high': [],
                    'Female_proportion': []
                   })

data_statistics = pd.DataFrame(data_statistics)

all_patients = []

model_type = 'prevalent'
control_data = 'HC'

for disease in ['AllCancers', 'Bladder', 'Breast', 'Colorectal', 'Kidney', 'Leukaemia', \
                'Lung', 'Lymphoma', 'Malignant_melanoma', 'Ovarian', 'Prostate', \
                'Thyroid', 'Uterine']:
    for data_type in ['prot', 'met', 'all', 'clinical']:
    #for data_type in ['prot']:



        data_path = '../data/Patients_final/' + disease + '/ukb_' + \
                    disease + '_' + data_type + '_' + model_type +'.csv'    
        data = pd.read_csv(data_path, index_col = 0)

        new_row = {'Disease': disease,
                    'Data': data_type,
                    'Count': len(data),
                    'Age_median': np.median(data['age_at_recruitment_f21022_0_0']),
                    'Age_IQR_low': np.quantile(data['age_at_recruitment_f21022_0_0'], 0.25),
                    'Age_IQR_high': np.quantile(data['age_at_recruitment_f21022_0_0'], 0.75),
                    'Female_proportion': sum(data['sex_f31_0_0'] == 'Female')/len(data)
                    }
                    
        data_statistics = data_statistics.append(new_row, ignore_index=True)
            
        all_patients = np.union1d(data.index, all_patients)
            
            


In [ ]:
data_statistics = data_statistics.replace('prot', 'proteomics')
data_statistics = data_statistics.replace('met', 'metabolomics')
data_statistics = data_statistics.replace('all', 'genomics')


In [ ]:
data_statistics.sort_values(by = 'Count')

In [ ]:
data_statistics.to_csv('Summary_statistics.csv', index = False)

In [ ]:


# Set up the data formats for the output
boxplot_data_full = ({'prob':[],
                'reference':[],
                'disease':[],
                'data_type':[],
                'Model': [],
                'control_data': [],
                'N': []
                })
boxplot_data_full = pd.DataFrame(boxplot_data_full)

Shiny_data = ({'prob':[],
                'reference':[],
               'Disease':[],
               'Data': [],
               'N':[],
               'AUC_train':[],
               'AUC_test':[],
               'Features':[],
               'Coef':[],
               'Model':[],
               'control_data': []
              })
Shiny_data = pd.DataFrame(Shiny_data)



organs = !ls ../data/Patients_final
for disease in ['AllCancers', 'Bladder', 'Breast', 'Colorectal', 'Kidney', 'Leukaemia', \
                'Lung', 'Lymphoma', 'Malignant_melanoma', 'Ovarian', 'Prostate', \
                'Thyroid', 'Uterine']:
#for disease in ['Prostate']:
    for data_type in ['prot', 'met', 'clinical']:
    #for data_type in ['prot']:

        if data_type == 'prot':
            omics = omics_prot.copy()
        elif data_type == 'met':
            omics = omics_met.copy()
            
        #for model_type in ['incident', 'prevalent']:   
        for model_type in ['prevalent']:   
            for control_data in ['HC', 'NonCancer']:
            #for control_data in ['HC']:


                if model_type == 'incident':
                    sick_path = '../data/Patients_final/' + disease + '/ukb_' + \
                                disease + '_' + data_type + '_incident.csv'    
                    clinical_sick = pd.read_csv(sick_path, index_col = 0)
                    HC_path = '../data/Patients_final/' + disease + '/ukb_' + \
                                disease + '_' + data_type + '_' + control_data + '_PairedToIncident.csv'
                    clinical_healthy = pd.read_csv(HC_path, index_col = 0)
                else:
                    sick_path = '../data/Patients_final/' + disease + '/ukb_' + \
                                disease + '_' + data_type + '_prevalent.csv'    
                    clinical_sick = pd.read_csv(sick_path, index_col = 0)
                    HC_path = '../data/Patients_final/' + disease + '/ukb_' + \
                                disease + '_' + data_type + '_' + control_data + '_PairedToPrevalent.csv'
                    clinical_healthy = pd.read_csv(HC_path, index_col = 0)


                # clinical -> clinical data from both healthy and sick
                # omics_clinical -> subset of metabolomics/proteomics including only patients from clinical data
                clinical = pd.concat([clinical_sick,clinical_healthy])
                
                if data_type == 'clinical':
                    omics = pd.get_dummies(clinical.drop('group', axis = 1))
                    scaler = MinMaxScaler()
                    omics = pd.DataFrame(scaler.fit_transform(omics), columns=omics.columns, index = omics.index)
                    

                omics_clinical = omics.loc[clinical.index.values]

                #remove proteins/metabolites with more than 10% NA
                omics_clinical = omics_clinical[omics_clinical.columns[omics_clinical.isna().sum() < np.shape(omics_clinical)[0]/10]]
                omics_clinical.replace([np.inf, -np.inf], np.nan, inplace=True)

                
                
                #devide between healthy and sick
                X_train, X_test, y_train, y_test = train_test_split(omics_clinical,
                                                                    clinical['group'],
                                                                    test_size=0.3,
                                                                    random_state=42)

                #KNN imputer(default parameters): train on train data, apply on both train and test
                imp = KNNImputer(n_neighbors=5, weights="uniform")
                imp.fit(X_train)
                X_train = pd.DataFrame(imp.transform(X_train), index = X_train.index, columns = X_train.columns)
                X_test = pd.DataFrame(imp.transform(X_test), index = X_test.index, columns = X_test.columns)



                # ExtraTreesClassifier(default parameters): Choose top 5 proteins/metabolites
                clf = ExtraTreesClassifier(n_estimators=10000,
                                           random_state=42,
                                           verbose = 1).fit(X_train, y_train)

                best_AUC = 0
                for N in range(1,16):
                #for N in range(4,6):
                    # Take only top N proteins/metabolites
                    model = SelectFromModel(clf, prefit=True, max_features = N)
                    X_train_subset = X_train[X_train.columns[model.get_support()]]
                    X_test_subset = X_test[X_test.columns[model.get_support()]]


                    #train ridge regression model

                    clf_ridge = LogisticRegressionCV(cv=10, Cs = 10, random_state=42, max_iter = 10000,
                                                penalty='l2').fit(X_train_subset, y_train)



                    #compute feature importance 
                    coefficients = clf_ridge.coef_[0]
                    feature_importance = pd.DataFrame({'Feature': X_train_subset.columns,
                                                        'Importance': np.abs(coefficients),
                                                        'Coef': coefficients})
                    feature_importance = feature_importance.sort_values('Importance', ascending = True)


                    #compute metrics
                    auc_train = roc_auc_score(y_train, clf_ridge.predict_proba(X_train_subset)[:, 1])
                    auc = roc_auc_score(y_test, clf_ridge.predict_proba(X_test_subset)[:, 1])
                    conf_matrix = confusion_matrix(y_test, clf_ridge.predict(X_test_subset), labels = np.unique(y_test))


                    #print summary
                    print('Disease: ' + disease)
                    print(conf_matrix)
                    print('AUC train: ' + str(auc_train))
                    print('AUC: ' + str(auc))
                    print('\n')




                    # Prediction for test dataset
                    X_test_summary = pd.DataFrame(clf_ridge.predict_proba(X_test_subset)[:, 1], columns = ['prob'])
                    X_test_summary['reference'] = y_test.values


                    # Data for boxplot
                            
                    boxplot_data = X_test_summary[['prob', 'reference']]
                    boxplot_data['disease'] = disease
                    boxplot_data['data_type'] = data_type
                    boxplot_data['Model'] = model_type
                    boxplot_data['control_data'] = control_data
                    boxplot_data['N'] = N
                        
                    if auc > best_AUC: 
                        boxplot_data_best = boxplot_data.copy()
                        best_AUC = auc

                    # Data for ShinyApp
                    new_row = {'prob': list(np.round(boxplot_data['prob'].astype(float), 3)),
                                'reference': list(boxplot_data['reference']),
                                'Disease': disease,
                                'Data': data_type,
                                'N': N,
                                'AUC_train': auc_train,
                                'AUC_test': auc,
                                'Features': list(feature_importance['Feature'].astype(str)),
                                'Coef': list(np.round(feature_importance['Coef'], 3)),
                                'Model': model_type,
                                'control_data': control_data
                        }
                    
                    Shiny_data = Shiny_data.append(new_row, ignore_index=True)
                print(best_AUC)
                boxplot_data_full = boxplot_data_full.append(boxplot_data_best, ignore_index=True)


In [ ]:
Shiny_data = Shiny_data[['Disease', 'Model', 'Data', 'control_data', 'N', 'AUC_train', 'AUC_test', 'Features', 'Coef', 'prob', 'reference']]
Shiny_data

In [ ]:
Shiny_data.to_csv('ShinyData_cancer_diagnosis.csv') #Supplementary table 2 (proteomics + metabolomics)
boxplot_data_full.to_csv('boxplot_data_cancer_diagnosis.csv')